# MovieApp - Kişiselleştirilmiş Film ve Dizi Öneri Sistemi
# Veri Analizi ve Model Geliştirme Notebook'u

## 1. Giriş ve Kütüphane İmportları

In [ ]:
# Gerekli kütüphanelerin import edilmesi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import os
import re
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import pickle
import warnings

# Metin işleme için
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk

# İşbirlikçi filtreleme için
from scipy.sparse.linalg import svds

# Görselleştirme ayarları
plt.style.use('seaborn-whitegrid')
sns.set_palette('deep')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
warnings.filterwarnings('ignore')

# NLTK kaynakları indirme
nltk.download('stopwords')
nltk.download('punkt')

print("Kütüphaneler başarıyla yüklendi!")

## 2. TMDB API'den Veri Çekme

In [ ]:
# API yapılandırması
TMDB_API_KEY = "YOUR_API_KEY"  # Gerçek uygulamada .env dosyasından alınmalı
BASE_URL = "https://api.themoviedb.org/3"

In [ ]:
# API'den film verisi çekme fonksiyonu
def fetch_movie_data(movie_id):
    """TMDB API'den film verisi çeker"""
    url = f"{BASE_URL}/movie/{movie_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "tr-TR",
        "append_to_response": "credits,keywords,videos"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Hata: {response.status_code}")
        return None

# API'den dizi verisi çekme fonksiyonu
def fetch_tv_data(tv_id):
    """TMDB API'den dizi verisi çeker"""
    url = f"{BASE_URL}/tv/{tv_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "tr-TR",
        "append_to_response": "credits,keywords,videos"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Hata: {response.status_code}")
        return None

# Popüler filmleri çekme
def fetch_popular_movies(page=1):
    """TMDB'den popüler filmleri çeker"""
    url = f"{BASE_URL}/movie/popular"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "tr-TR",
        "page": page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()['results']
    else:
        print(f"Hata: {response.status_code}")
        return []

# Popüler dizileri çekme
def fetch_popular_tv(page=1):
    """TMDB'den popüler dizileri çeker"""
    url = f"{BASE_URL}/tv/popular"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "tr-TR",
        "page": page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()['results']
    else:
        print(f"Hata: {response.status_code}")
        return []